In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### 分层索引

In [2]:
data = pd.Series(np.random.randn(9),
                 index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data  # 具有两层索引的Series

a  1   -0.929424
   2   -0.215367
   3    1.505062
b  1   -0.383148
   3   -0.277732
c  1   -0.657639
   2   -1.218261
d  2   -0.619204
   3   -0.017534
dtype: float64

In [3]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [4]:
data['b']        # 使用“外层”索引

1   -0.383148
3   -0.277732
dtype: float64

In [5]:
data['b':'d']    # 使用“外层”索引

b  1   -0.383148
   3   -0.277732
c  1   -0.657639
   2   -1.218261
d  2   -0.619204
   3   -0.017534
dtype: float64

In [6]:
data[['b','d']]  # 使用“外层”索引

b  1   -0.383148
   3   -0.277732
d  2   -0.619204
   3   -0.017534
dtype: float64

In [7]:
data.loc[:, 2]   # 使用“内层”索引

a   -0.215367
c   -1.218261
d   -0.619204
dtype: float64

### 层次化索引在数据重塑和基于分组的操作（如透视表生成）中扮演着重要的角色

In [8]:
data.unstack()

,1,2,3
a,-0.929424,-0.215367,1.505062
b,-0.383148,NaN,-0.277732
c,-0.657639,-1.218261,NaN
d,NaN,-0.619204,-0.017534


In [9]:
data.unstack().stack()

a  1   -0.929424
   2   -0.215367
   3    1.505062
b  1   -0.383148
   3   -0.277732
c  1   -0.657639
   2   -1.218261
d  2   -0.619204
   3   -0.017534
dtype: float64

In [10]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns=[['Ohio', 'Ohio', 'Colorado'],
                              ['Green', 'Red', 'Green']])
frame  # DataFrame的每一个轴上都可以拥有分层索引

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [11]:
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame  # DataFrame的每一个层级的索引都可以有一个名称

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [12]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [13]:
# 可以单独创建MultiIndex对象然后复用
pd.MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']], names=['state', 'color'])

MultiIndex([(    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('Colorado', 'Green')],
           names=['state', 'color'])

### 重新排序与层级排序

In [14]:
frame.swaplevel('key1', 'key2')  # swaplevel()接受两个层级编号或名称，并返回一个互换了层级的新对象

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [15]:
frame.swaplevel(0, 1).sort_index(level=0)  # sort_index()是根据单个层级中的值对数据进行排序

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

### 按层级进行汇总统计

In [16]:
frame.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [17]:
frame.sum(level='color', axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

### 使用DataFrame的列进行索引

In [18]:
frame = pd.DataFrame({'a': range(7),
                      'b': range(7, 0, -1),
                      'c': ['one', 'one', 'one', 'two', 'two', 'two', 'two'],
                      'd': [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [19]:
frame2 = frame.set_index(['c', 'd'])  # set_index()会将其一个或多个列转换为行索引，并创建一个新的DataFrame
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [20]:
frame.set_index(['c', 'd'], drop=False)  # 默认情况下，那些列会从DataFrame中移除，但也可以将其保留下来

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [21]:
frame2.reset_index()  # reset_index()的功能跟set_index()刚好相反，层次化索引的级别会被转移到列里面

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1
